# **IAA - PRÀCTICA: MAIN**

### **Instal·lar llibreries necessàries**

In [ ]:
%pip install -r ../assets/requirements.txt 

### **Importar llibreries**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl

### **Llegir les dades (Cirrhosis Dataset)**

In [ ]:
def load_dataset(save_to_csv: bool = True):
	global data
	from ucimlrepo import fetch_ucirepo 
	
	# Fetch dataset
	cirrhosis_patient_survival_prediction = fetch_ucirepo(id=878)

	data = pd.DataFrame(cirrhosis_patient_survival_prediction.data.original)

	if save_to_csv:
		# Guardem el dataset per poder-lo visualitzar sencer
		data.to_csv('../assets/data/raw_cirrhosis.csv')

load_dataset(save_to_csv=True)

### **Informació del dataset inicial**

In [ ]:
data.shape

In [ ]:
data.head(-10)

In [ ]:
data.info()

### **Preprocessing inicial**

In [ ]:
def initial_preprocessing(data: pd.DataFrame, save_to_csv: bool = True):
	"""
	Reemplaça els valors 'NaNN' per NaN, assigna els tipus de dades correctes a cada columna i renombra les classes d'algunes variables per una millor comprensió.
	"""
	# Reemplaçar l'string 'NaNN' per NaN
	data.replace(to_replace=['NaNN', '', pd.NA], value=np.nan, inplace=True)

	# Assignem els tipus de dades correctes a cada columna
	int64_variables = ['N_Days', 'Age', 'Cholesterol', 'Copper', 'Tryglicerides', 'Platelets']
	float64_variables = ['Bilirubin', 'Albumin', 'Alk_Phos', 'SGOT', 'Prothrombin']
	category_variables = ['ID', 'Status', 'Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema', 'Stage']

	data[int64_variables] = data[int64_variables].astype('Int64')
	data[float64_variables] = data[float64_variables].astype('float64')
	data[category_variables] = data[category_variables].astype('category')

	# Renombrem les classes d'algunes variables per una millor comprensió
	data['Status'] = data['Status'].replace({'D': 'Dead', 'C': 'Alive', 'CL': 'Liver Transplant'})

	if save_to_csv:
		# Guardem el dataset
		data.to_csv('../assets/data/initial_preprocessing_cirrhosis.csv')

initial_preprocessing(data=data, save_to_csv=True)

In [ ]:
data.head(-10)

### **Anàlisis de les variables**

In [ ]:
data.head(-10)

In [ ]:
data.isna().sum().sort_values(ascending=False)

In [ ]:
# Estudi de les variables numèriques
data.describe()

In [ ]:
# Estadístiques de les variables categòriques
data.describe(include='category')

In [ ]:
def numerical_vars_histograms(data: pd.DataFrame):
    # Visualització de les distribucions de les variables numèriques en una sola figura
    numerical_columns = data.select_dtypes(include=['Int64', 'float64']).columns

    num_rows = int(np.ceil(len(numerical_columns) / 2))

    fig = plt.figure(figsize=(10, num_rows * 4))

    for i, col in enumerate(numerical_columns):
        ax = fig.add_subplot(num_rows, 2, i + 1)
        
        sns.histplot(data[col], edgecolor="k", linewidth=1.5, kde=True)
        
        plt.xticks(rotation=45, ha='right')
        
        ax.set_title(f'Distribució de la variable numèrica {col}')
        ax.set_xlabel(col)
        ax.set_ylabel('Freqüència')

    plt.tight_layout()
    plt.show()

numerical_vars_histograms(data=data)

In [ ]:
def categorical_vars_countplots(data: pd.DataFrame):
    """
    Visualització de les distribucions de les variables categòriques en una sola figura (menys ID).
    """
    # Visualització de les distribucions de les variables categòriques en una sola figura (menys ID)
    categorical_columns = data.select_dtypes(include=['category']).columns.drop(['ID'])
    num_rows = int(np.ceil(len(categorical_columns) / 2))

    fig = plt.figure(figsize=(10, num_rows * 4))

    for i, col in enumerate(categorical_columns):
        ax = fig.add_subplot(num_rows, 2, i + 1)
        
        sns.countplot(data=data, x=col, ax=ax, hue=col, legend=False)
        
        plt.xticks(rotation=45, ha='right')
        
        ax.set_title(f'Distribució de la variable categòrica {col}')
        ax.set_xlabel(col)
        ax.set_ylabel('Quantitat')

    plt.tight_layout()
    plt.show()

categorical_vars_countplots(data=data)

### **Tractament d'outliers**

In [ ]:
def compare_iqr_factors(data: pd.DataFrame, factors: list = [1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]):
	"""
	Compara diferents factors que multipliquen al IQR per a determinar els outliers i realitza un gràfic evolutiu per comparar-los.
	"""
	numerical_columns = data.select_dtypes(include=['Int64', 'float64']).columns

	plt.figure(figsize=(10, 6))

	# Dictionary to store outlier percentages for each factor and column
	outlier_percentages = {col: [] for col in numerical_columns}
	total_percentages = [set() for _ in range(len(factors))]

	for col in numerical_columns:
		Q1 = data[col].quantile(0.25)
		Q3 = data[col].quantile(0.75)
		IQR = Q3 - Q1

		for f_id, factor in enumerate(factors):
			outliers_mask = ((data[col] < (Q1 - factor * IQR)) | (data[col] > (Q3 + factor * IQR)))
			total_percentages[f_id].update(data.index[outliers_mask])
			outliers_percentage = np.mean(outliers_mask) * 100
			outlier_percentages[col].append(outliers_percentage)

	total_percentages = [(len(outliers) / len(data)) * 100 for outliers in total_percentages]
			
	# Plotting the results
	for col, percentages in outlier_percentages.items():
		plt.plot(factors, percentages, label=col)
	
	plt.plot(factors, total_percentages, label='Total', linestyle='--', color='black')
	plt.xlabel('Factor multiplicatiu del IQR')
	plt.ylabel('Percentage d\'outliers (%)')
	plt.title('Percentatge d\'outliers de cada variable numèrica per a diferents factors multiplicatius del IQR')
	plt.legend()
	plt.grid(True)
	plt.show()

compare_iqr_factors(data=data, factors=[1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5])

In [ ]:
def delete_outliers(data: pd.DataFrame, factor: float = 1.5, plots: bool = True):
    """
    Funció que detecta, visualitza i elimina els outliers d'un dataset. El factor multiplica el IQR per a determinar quins valors són outliers.
    """
    # Detecció, visualització i eliminació d'outliers
    numerical_columns = data.select_dtypes(include=['Int64', 'float64']).columns

    outliers_indices = []

    for col in numerical_columns:
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1
        outliers_mask = ((data[col] < (Q1 - factor * IQR)) | (data[col] > (Q3 + factor * IQR)))
        outliers = data[col][outliers_mask]
        non_outliers = data[col][~outliers_mask]

        outliers_indices.extend(data[col][outliers_mask].index.tolist())
        
        if plots:
            fig, axes = plt.subplots(1, 2, figsize=(8, 5))

            # Boxplot amb els outliers originals
            sns.boxplot(ax=axes[0], y=data[col], orient='v')
            axes[0].scatter(x=[0]*len(outliers), y=outliers, color='red', marker='o')
            axes[0].set_title(f'Boxplot de {col} amb outliers ({factor}x IQR)')
            
            percent_outliers = len(outliers) / data.shape[0] * 100
            axes[0].text(0.5, -0.1, f'Outliers ({factor}x IQR): {len(outliers)} ({percent_outliers:.2f}%)', 
                        ha='center', va='center', transform=axes[0].transAxes)
            # Boxplot sense els outliers
            sns.boxplot(ax=axes[1], y=non_outliers, orient='v')
            axes[1].set_title(f'Boxplot de {col} sense outliers')

            plt.tight_layout()
            plt.show()

    unique_outliers = len(set(outliers_indices))
    print(f"Nombre total d'outliers únics eliminats: {unique_outliers} ({unique_outliers / data.shape[0] * 100:.2f}% de tot el dataset)")

    # Eliminació d'outliers
    data = data.drop(list(set(outliers_indices)))

delete_outliers(data=data, factor=3, plots=True)

### **Recodificació de variables**

In [ ]:
def encode_variables(data: pd.DataFrame, save_to_csv: bool = True):
	"""
	Codifica les variables categòriques per a poder-les utilitzar en els models de ML.
	"""
	# Codificació de les variables categòriques
	from sklearn.preprocessing import LabelEncoder

	le = LabelEncoder()

	categorical_columns = data.select_dtypes(include=['category']).columns

	for col in categorical_columns:
		data[col] = le.fit_transform(data[col])

	if save_to_csv:
		# Guardem el dataset
		data.to_csv('../assets/data/encoded_cirrhosis.csv')

### **Partició del dataset en train/test**

In [29]:
def split_dataset(data: pd.DataFrame, test_size: float = 0.15, random_state: int = 42):
	"""
	Particiona el dataset en train i test.
	"""
	global X_train, y_train, test

	from sklearn.model_selection import train_test_split

	train, test = train_test_split(data, test_size=test_size, random_state=random_state)

	print(f"Train: {train.shape}")
	print(f"Test: {test.shape}")

	# 'Status' és la variable target
	X_train = train.drop(columns=['Status'])
	y_train = train['Status']

split_dataset(data=data, test_size=0.15, random_state=42)

Train: (355, 20)
Test: (63, 20)
Status
Alive               202
Dead                132
Liver Transplant     21
Name: count, dtype: int64


### **Imputar els valors faltants (Missings)**

In [ ]:
def impute_missings(data: pd.DataFrame, save_to_csv: bool = True):
    from sklearn.impute import KNNImputer
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.compose import ColumnTransformer

    categorical_columns = data.select_dtypes(include=['category']).columns.drop(['ID'])

    # Creación del transformador para la codificación One-Hot
    one_hot_encoder = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns)
        ],
        remainder='passthrough'
    )

    # Aplicación de la codificación One-Hot
    data_encoded = one_hot_encoder.fit_transform(data)

    # Creación del imputador KNN
    knn_imputer = KNNImputer(n_neighbors=5)

    # Imputación de los datos codificados
    data_imputed = knn_imputer.fit_transform(data_encoded)

    # Convirtiendo de nuevo a un DataFrame de pandas (opcional, dependiendo de cómo necesites usar los datos)
    # Nota: La conversión de nuevo a DataFrame requiere asignar nombres a las columnas
    # Esto puede ser complejo debido a la codificación One-Hot
    data_imputed_df = pd.DataFrame(data_imputed)

    # Mostrando las primeras filas de los datos imputados
    data_imputed_df.head()

    if save_to_csv:
        # Guardem el dataset
        data_imputed_df.to_csv('../assets/data/imputed_cirrhosis.csv')


In [ ]:
from sklearn.impute import SimpleImputer
num_columns = X_train.select_dtypes(include=['float64', 'int64']).columns
cat_columns = X_train.select_dtypes(include=['object']).columns
imputer_num = SimpleImputer(strategy='median')
imputer_cat = SimpleImputer(strategy='most_frequent')
X_train[num_columns] = imputer_num.fit_transform(X_train[num_columns])
X_train[cat_columns] = imputer_cat.fit_transform(X_train[cat_columns])
X_test[num_columns] = imputer_num.transform(X_test[num_columns])
X_test[cat_columns] = imputer_cat.transform(X_test[cat_columns])

# Codificación one-hot de variables categóricas
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
X_train_encoded = pd.DataFrame(encoder.fit_transform(X_train[cat_columns]))
X_train_encoded.columns = encoder.get_feature_names_out(cat_columns)
X_train = X_train.drop(cat_columns, axis=1)
X_train = pd.concat([X_train, X_train_encoded], axis=1)
X_test_encoded = pd.DataFrame(encoder.transform(X_test[cat_columns]))
X_test_encoded.columns = encoder.get_feature_names_out(cat_columns)
X_test = X_test.drop(cat_columns, axis=1)
X_test = pd.concat([X_test, X_test_encoded], axis=1)

**1r Model: K-Nearest Neighbors (KNN)**

**2n Model: Decision Tree**

**3r Model: Support Vector Machine (SVM)**